# Toy Problem Optimization

This notebook explores what happens when you take the two-link rope toy problem and pull the rope to the right

In [4]:
import toy_problem_optimization_common

ImportError: No module named 'toy_problem_optimization_common'

### Define goal

Our goal will be denoted as $g$, for now we simply assume its defined in the full state space

In [2]:
g = np.array([[5], [0], [6], [0]])

# Manual Data

Let's pretend our data comes from pulling a 2-link object to the right for 6 time steps

In [5]:
manual_data = []
for _ in range(100):
    y = 0 #np.random.randint(-10,10)
    i = np.random.randint(-5,5)
    # Training data looks like [(s_t, u_t, s_{t+1}, c_t, c_{t+1})]
    manual_data.append((np.array([[i],[y],[i+1],[y]]),
         np.array([[1],[0]]),
         np.array([[i+1],[y],[i+2],[y]]),
         np.array([[(g[0] - i)**2]]),
         np.array([[(g[0] - i+1)**2]])))

### How well do Random Parameters Do (on average)

In [6]:
N = 1000
costs = np.zeros(N)
for i in range(N):
    random_params = np.random.randn(8)
    costs[i] = latent_prediction_objective(random_params, data=manual_data, alpha=0.5)
print("Median cost for random parameters {:.3f}".format(np.median(costs)))

Median cost for random parameters 729.557


### Test Objective Function

To double check my objective function, let's make sure that my hand-designed model reductions give zero in the objective function:

In [7]:
my_params = np.array([1, 0, 0, 0, 0, 1, 0, 1])
print("loss for hand-designed good params:", latent_prediction_objective(my_params, data=manual_data, alpha=0.5))

my_params = np.array([0, 1, 0, 0, 0, 1, 0, 1])
print("loss for hand-designed bad params:", latent_prediction_objective(my_params, data=manual_data, alpha=0.5))

loss for hand-designed good params: 0.0
loss for hand-designed bad params: 623.2375


In [8]:
params = train(manual_data, latent_prediction_objective, alpha=0.5)

Finished in 5 iterations


In [9]:
print("Objective Cost/Loss:", latent_prediction_objective(params, manual_data, alpha=0.5))
A, B, C, D = params_to_matrices(params)
print("Model reduction Matrix:", A)
print("Dynamics Matrix:", B, ',', C)
print("Cost Matrix:", D)

Objective Cost/Loss: 4.281324573080723e-25
Model reduction Matrix: [[ 0.03 13.3   0.04 12.62]]
Dynamics Matrix: [[-0.]] , [[ 0.06 12.91]]
Cost Matrix: [[244.77]]


### We can now perfectly predict future latent state and cost

Consider what happens if we follow a plan of pulling right from the origin
What is the predicted cost of the plan $[[1,0], \dots]$ ? In the real world, the sum of cost at each state is $5^2+4^2+3^2+2^2+1^2=55$

In [10]:
actions = [np.array([[1],[0]]), np.array([[1],[0]]),np.array([[1],[0]]),np.array([[1],[0]]),np.array([[1],[0]])]
s0 = np.array([[0], [0], [1], [0]])
o = A@s0
predicted_total_cost = 0
for i, u in enumerate(actions):
    o_ = o + B@o + C@u
    c_hat = (A@g - o).T@D@(A@g - o)
    predicted_total_cost += c_hat
    o = o_
print(predicted_total_cost)

[[55.]]


Ok great! **We can perfectly predict the future latent state and cost!**

### What if we only consider cost, and predicting cost?

In [12]:
my_params = np.array([1, 0, 0, 0, 0, 1, 0, 1])
print("loss for hand-designed good params:",
      one_step_cost_prediction_objective(my_params, data=manual_data, alpha=0.5))

my_params = np.array([0, 1, 0, 0, 0, 1, 0, 1])
print("loss for hand-designed bad params:",
      one_step_cost_prediction_objective(my_params, data=manual_data, alpha=0.5))

loss for hand-designed good params: 0.0
loss for hand-designed bad params: 623.2375


In [13]:
params = train(manual_data, one_step_cost_prediction_objective)

Finished in 8 iterations


In [14]:
print("Prediction Objective:", latent_prediction_objective(params, manual_data, alpha=0.5))
print("Cost-Only Objective:", one_step_cost_prediction_objective(params, manual_data, alpha=0.5))
A, B, C, D = params_to_matrices(params)
print("Model reduction Matrix:", A)
print("Dynamics Matrix:", B, ',', C)
print("Cost Matrix:", D)

Prediction Objective: 1.5213205860939539e-27
Cost-Only Objective: 3.2808697006601915e-27
Model reduction Matrix: [[   0.22 5324.04   -0.22 5322.36]]
Dynamics Matrix: [[-0.]] , [[  -0.   5323.29]]
Cost Matrix: [[171375.06]]


Unsurprisingly, we are also able to fit our data in this case. Much more interestingly, **We also get latent state prediction accuracy for free, just by predicting in cost!**

# What if the data is more diverse?

What if we include various y values?

In [15]:
manual_data = []
for _ in range(100):
    y = np.random.randint(-10,10)
    i = np.random.randint(-5,5)
    manual_data.append((np.array([[i],[y],[i+1],[y]]),
         np.array([[1],[0]]),
         np.array([[i+1],[y],[i+2],[y]]),
         np.array([[(g[0] - i)**2]]),
         np.array([[(g[0] - i+1)**2]])))

In [16]:
my_params = np.array([1, 0, 0, 0, 0, 1, 0, 1])
print("loss for hand-designed good params:", latent_prediction_objective(my_params, data=manual_data, alpha=0.5))

my_params = np.array([0, 1, 0, 0, 0, 1, 0, 1])
print("loss for hand-designed bad params:", latent_prediction_objective(my_params, data=manual_data, alpha=0.5))

loss for hand-designed good params: 0.0
loss for hand-designed bad params: 465.9475


In [17]:
params = train(manual_data, latent_prediction_objective)

Finished in 10 iterations


In [18]:
print("Objective Cost/Loss:", latent_prediction_objective(params, manual_data, alpha=0.5))
A, B, C, D = params_to_matrices(params)
print("Model reduction Matrix:", A)
print("Dynamics Matrix:", B, ',', C)
print("Cost Matrix:", D)

Objective Cost/Loss: 2.612220840338469e-20
Model reduction Matrix: [[-1.92 -0.51  1.21  0.51]]
Dynamics Matrix: [[0.]] , [[-0.71 26.03]]
Cost Matrix: [[1.97]]


Yup, still works. Although this data is not realistic to the motion of the links. This just shows that if we add a distractor variable which does not help explain the cost, we can still "ignore it". Specifically, the B matrix (just a real number here) is still 0 and the y components of the model reduction can be arbitrary values.

# Now with some data from Gazebo

In [34]:
new_data = load_gazebo_data("/home/pmitrano/catkin_ws/src/link_bot/link_bot_teleop/data/fwd_1.txt")

In [1]:
plot_gz_data(new_data)

NameError: name 'plot_gz_data' is not defined

### See how our previous model reduction transfers

To use the same models, the dimensionality of the input data must be the same.

In [36]:
print("Objective Cost/Loss transfering to more complicated simulation:",
      latent_prediction_objective(params, new_data, alpha=0.5))

Objective Cost/Loss transfering to more complicated simulation: 1.3686914577862098e-06


### Show that our hand-designed parameters still work well

In [37]:
my_params = np.array([1, 0, 0, 0, 0, 0.1, 0, 1])
print("Objective Cost/Loss:", latent_prediction_objective(my_params, new_data, alpha=0.5))

Objective Cost/Loss: 2.6472522881450943e-05


### Try to optimize on our new data

In [38]:
params = train(new_data, latent_prediction_objective)

Finished in 32 iterations


In [39]:
print("Objective Cost/Loss:", latent_prediction_objective(params, new_data, alpha=0.5))
A, B, C, D = params_to_matrices(params)
print("Model reduction Matrix:", A)
print("Dynamics Matrix:", B, ',', C)
print("Cost Matrix:", D)

Objective Cost/Loss: 1.2712319581078583e-05
Model reduction Matrix: [[  0.25 189.48  -0.32 421.56]]
Dynamics Matrix: [[0.]] , [[-0.01 -0.01]]
Cost Matrix: [[212.84]]


### Performance on Gazebo data may not be quite as good

So there are two issues here. First, the control equation is not actually linear anymore. This is because in this gazebo data the links are accelerating, so their delta x position cannot be computed as $v_x*\Delta t$ anymore. This would explain why the hand specified parameters do not give as small of a loss (1e-5 versus 1e-20)

There may be another issue which is that the optimizer finds are not as good as the manual parameters anymore. However, I cannot confirm whether this issue is statistically significant.